## Imports

In [2]:
import pathlib
from super_resolution.src.sen2venus_dataset import create_train_validation_test_split

## Constants

In [3]:
DATA_DIR = pathlib.Path("C:/Users/Mitch/stat3007_data")
SITES_DIR = DATA_DIR / "sites"

In [5]:
train_patches, val_patches, test_patches = create_train_validation_test_split(
    str(SITES_DIR) + "\\", seed=42
)